In [1]:
import os

In [2]:
os.chdir('../..')

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
import dgl
import faiss

from dgl.nn.pytorch.conv import SAGEConv

Using backend: pytorch


In [4]:
dataset = DglNodePropPredDataset(name='ogbn-arxiv', 
                                 root='data/dataset_dgl/')
dataset

DglNodePropPredDataset(1)

In [5]:
torch.cuda.is_available()

True

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [7]:
graph = dataset[0][0]
graph

Graph(num_nodes=169343, num_edges=1166243,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [8]:
torch.mean(graph.out_degrees() + graph.in_degrees() * 1.)

tensor(13.7737)

## Build Graph from deepwalk embedding distances

In [9]:
output_emb_file = 'models/deepwalk_05_trvlts_embedding.npy'
k_closest = 25

In [10]:
embeddings = np.load(output_emb_file)

In [11]:
index = faiss.IndexFlatL2(embeddings.shape[1])

In [12]:
index.train(embeddings)
index.add(embeddings)

In [13]:
distances, indices = index.search(embeddings, k_closest + 1)

In [14]:
indices

array([[     0,  36054,  79872, ..., 148781,  92115,  71488],
       [     1, 154949, 100594, ..., 123921,  86016,  64576],
       [     2, 113232,  14632, ..., 157276, 125733, 159747],
       ...,
       [169340,  61320, 103381, ..., 102517,  14350,  54841],
       [169341,  34227,  73771, ...,   4684,  61526,  92611],
       [169342, 127946,  17652, ..., 143761,  39588,   1461]])

In [15]:
u = np.repeat(indices[:, 0], k_closest)
u.shape

(4233575,)

In [16]:
v = indices[:, 1:].reshape(-1)
v.shape

(4233575,)

In [17]:
graph.remove_edges(graph.edge_ids(graph.edges()[0], graph.edges()[1]))

In [18]:
graph.edges()

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [19]:
graph.add_edges(u, v)

In [20]:
graph.edges()

(tensor([     0,      0,      0,  ..., 169342, 169342, 169342]),
 tensor([ 36054,  79872, 153376,  ..., 143761,  39588,   1461]))

## Model

In [21]:
split_idx = dataset.get_idx_split()

In [22]:
torch.cuda.set_device(device)

In [23]:
features = graph.ndata['feat'].cuda()
labels = dataset.labels.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [24]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

<ipython-input-24-fa10932ea454>:1: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  train_nid = train_mask.nonzero().squeeze()


In [25]:
n_edges = graph.number_of_edges()

In [26]:
graph = graph.int().to(device)

In [27]:
n_iters = 3000
epochs = 1000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001

In [28]:
class ThreeLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim, dropout):
        super(ThreeLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(SAGEConv(input_dim, hidden_channels, 'mean'))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, hidden_channels, 'mean'))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, output_dim, 'mean'))
        
        self.dropout = dropout
        

    def forward(self, graph, x):
        x = self.convs[0](graph, x)
        x = self.bns[0](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[1](graph, x)
        x = self.bns[1](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[2](graph, x)
        return x.log_softmax(dim=-1)

In [29]:
model = ThreeLayer(input_dim,
                   hidden_channels,
                   output_dim,
                   0.5).cuda()

In [30]:
def train(model, graph, features, train_mask, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(graph, features)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [31]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator):
    model.eval()

    out = model(graph, features)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [32]:
evaluator = Evaluator(name='ogbn-arxiv')

In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, graph, features, train_mask, optimizer)
    result = test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.2869, Train: 68.47%, Valid: 67.73% Test: 65.56%
Epoch: 200, Loss: 1.1133, Train: 70.81%, Valid: 69.36% Test: 67.78%
Epoch: 300, Loss: 1.0295, Train: 72.15%, Valid: 70.07% Test: 68.58%
Epoch: 400, Loss: 0.9778, Train: 73.29%, Valid: 70.75% Test: 69.06%
Epoch: 500, Loss: 0.9339, Train: 74.12%, Valid: 70.98% Test: 69.42%
Epoch: 600, Loss: 0.8967, Train: 75.00%, Valid: 71.17% Test: 69.62%
Epoch: 700, Loss: 0.8724, Train: 75.78%, Valid: 71.21% Test: 69.61%
Epoch: 800, Loss: 0.8411, Train: 76.56%, Valid: 71.25% Test: 69.82%
Epoch: 900, Loss: 0.8173, Train: 77.23%, Valid: 71.22% Test: 69.77%
Epoch: 1000, Loss: 0.7943, Train: 77.96%, Valid: 71.13% Test: 69.54%
